In [3]:
import os
import requests
import pandas as pd

In [4]:
GITHUB_TOKEN = os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN")  
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

In [ ]:
stars = "500..1000"
forks = ">100"
pushed = ">2025-02-27"
created = "<2013-01-01"
keyword = "lib"
in_qualifiers = ""

query = f"{keyword} in: {in_qualifiers} created:{created} stars:{stars} forks:{forks} pushed:{pushed} mirror:false"
url = "https://api.github.com/search/repositories"
params = {
    "q": query,
    "sort": "stars",
    "order": "desc",
    "per_page": 100  # Max 100
}

In [6]:
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    items = response.json()["items"]
    data = [
        {
            "Repo Name": item["full_name"],
            "Stars": item["stargazers_count"],
            "Forks": item["forks_count"],
            "Link": item["html_url"]
        }
        for item in items
    ]
    df = pd.DataFrame(data)
    print(f"Collected {df.shape[0]} repositories")
else:
    print(f"Error {response.status_code}: {response.json()}")

Collected 8 repositories


In [7]:
df

,Repo Name,Stars,Forks,Link
0,lpereira/hardinfo,779,131,https://github.com/lpereira/hardinfo
1,rime/ibus-rime,774,106,https://github.com/rime/ibus-rime
2,libplctag/libplctag,743,240,https://github.com/libplctag/libplctag
3,uroni/urbackup_backend,704,150,https://github.com/uroni/urbackup_backend
4,tcltk/tcl,702,198,https://github.com/tcltk/tcl
5,cping/LGame,667,172,https://github.com/cping/LGame
6,gregkh/kernel-development,631,128,https://github.com/gregkh/kernel-development
7,rhash/RHash,604,121,https://github.com/rhash/RHash


In [ ]:
l